In [11]:
import pandas as pd
import numpy as np
import re
pd.set_option("display.max_columns", None)
pd.reset_option('display.max_colwidth', 100)

In [2]:
ogair_df = pd.read_csv('C:/Users/rubyc/Documents/work/ebfd/requests_and_consultations_airtable_92724.csv')
ogair_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421 entries, 0 to 420
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   Name                                  421 non-null    object
 1   Date Called                           418 non-null    object
 2   Date Called Back                      26 non-null     object
 3   Call To:                              417 non-null    object
 4   Contact Info                          402 non-null    object
 5   County                                30 non-null     object
 6   Client Name if Different from Caller  67 non-null     object
 7   Referral From (Agency/Individual)     294 non-null    object
 8   QLSP?                                 402 non-null    object
 9   Non-QLSP Type                         74 non-null     object
 10  Message                               382 non-null    object
 11  Notes                           

# 1. Data Cleaning

##1.1 Null Values
**Purpose:** Check for the proportion of null values in the data and
<br>**Code Breakdown by line:**
<br>1. Uses Pandas built-in isnull function to check for null values and calculate the number against the total number of values in the dataframe
<br> 2. Makes a dataframe of these proportions we just calculated
<br> 3. Sorts them by the highest to lowest proportion
<br> 4. Calls the dataframe we just made
<br> 5. Replaces all null values (which python reads as NaN) with the string "None"
<br>**Notes:** None this is very simple and obligatory. We chose to replace NaN with the string "None" bc most of what we're doing here is natural language processing, which utelizes functions that are designed to have a strings input and tend to output errors when they're input NaN instead.

In [3]:
null_percentage = ogair_df.isnull().sum() / len(ogair_df)
missing_value_df = pd.DataFrame({'proportion_null': null_percentage})
missing_value_df = missing_value_df.sort_values('proportion_null', ascending = False)
missing_value_df.head(30)

,proportion_null
Date Called Back,0.938242
Court Type,0.938242
County,0.928741
Client Name if Different from Caller,0.840855
Non-QLSP Type,0.824228
Referral From (Agency/Individual),0.301663
Conflict Check Completed,0.159145
Intake Form Completed,0.156770
Time Spent on Meeting,0.137767
Resolution,0.109264


In [4]:
nullfree_df = ogair_df.dropna(subset=['Date Called','QLSP?']).fillna('None')
nullfree_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, 0 to 420
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   Name                                  400 non-null    object
 1   Date Called                           400 non-null    object
 2   Date Called Back                      400 non-null    object
 3   Call To:                              400 non-null    object
 4   Contact Info                          400 non-null    object
 5   County                                400 non-null    object
 6   Client Name if Different from Caller  400 non-null    object
 7   Referral From (Agency/Individual)     400 non-null    object
 8   QLSP?                                 400 non-null    object
 9   Non-QLSP Type                         400 non-null    object
 10  Message                               400 non-null    object
 11  Notes                                

In [10]:

test_df = nullfree_df.head(5)
test_df.head()

,Name,Date Called,Date Called Back,Call To:,Contact Info,County,Client Name if Different from Caller,Referral From (Agency/Individual),QLSP?,Non-QLSP Type,Message,Notes,Court Type,Time Spent on Meeting,Intake Form Completed,Conflict Check Completed,Resolution,Created By
0,Aaron Belcher,9/12/2023,None,Meredith Cell,None,None,None,Fremont private school,No,None,CPS cops came and grabbed 17 year old and took...,"9/12 called....9/13 called back, gave info aob...",None,None,not applicable,not applicable,TA,Anonymous
1,Melissa Howell,9/13/2023,None,LSPC Main Line,"650-201-6262, Howellfamily015@gmail.com",None,None,Search Engine,No,None,Her husband died in prison. Needs to know what...,9/14 Meredith called back after speaking with ...,None,None,not applicable,not applicable,information or referral,Arielle Reisman
2,Brigitte Nicoletti,9/14/2023,None,"Arielle Cell,Meredith Cell",colleague,None,Tahtanerrisha Anecce Sessoms-Howell (Tahti),EBCLC,Yes,None,Brigitte has potential client referral.,Client currently living in hotel on San Pablo-...,None,.5,not applicable,not applicable,closed - no client follow up,Arielle Reisman
3,Brigitte Nicoletti,9/14/2023,None,"Meredith Cell,Arielle Cell",colleague,None,Dani,EBCLC,Yes,None,Need to get a client diapers and baby supplies...,Janette Lema- Hively Family Resource Center i...,None,.5,not applicable,not applicable,information or referral,Arielle Reisman
4,Terry Mcollumn,9/15/2023,None,LSPC Main Line,(567)-312-0321,None,None,None,No,None,He is looking for support with a child support...,Called - asked if I had reached Terry. Was tol...,None,None,not applicable,not applicable,closed - no client follow up,Arielle Reisman


# 2. Parsing out the data

##2.1 Callback Dates/Date Processing
**Purpose:** Extract the date that the call was returned from the "Notes" section and convert the extracted callback date and native call recieved date
<br>**Input:** The name of a dataframe, the column that the noted dates are in, and the column that the full call cates are in
<br>**Output:** A list of the callback dates in datetime format
<br>**Code Breakdown:**
1.  Creates empty list to later fill
2. Two regular expressions are defined as "yearpattern" and "datepattern" - yearpattern identifies four numerical digits in a row and datepattern identifies numbers 1-12 immediately followed by a slash immediately followed by numbers 1-31. I have no idea how regular expression syntax works those were generated w Chatgpt
3. Opens a for loop that'll go through every row in the column that you've defined as containing the callback dates, defining the contents of each row as "note"
  1. Within the for loop, we search the contents of the
5. Replaces all null values (which python reads as NaN) with the string "None"
**Notes:** None this is very simple and obligatory. We chose to replace NaN with the string "None" bc most of what we're doing here is natural language processing, which utelizes functions that are designed to have a strings input and tend to output errors when they're input NaN instead.

In [12]:
def pull_callback_date(df, datecol, yearcol):
  #designed for input: nullfree_df, Notes, Date Called
  callback_date_l = []
  yearpattern = r'\d{4}'
  datepattern = r'\b(0?[1-9]|1[0-2])/(0?[1-9]|[12][0-9]|3[01])\b'
  for note in df[datecol]:
    #  try:
    datefind = re.search(datepattern, note)
    if datefind:
      callbackdate = datefind.group()
      for date in df[yearcol]:
        yearfind = re.search(yearpattern, date)
      wholedate = (callbackdate + "/" + yearfind.group())
      final_datetime = pd.to_datetime(wholedate)
      callback_date_l.append(final_datetime)
    else: callback_date_l.append('None')
      #except Exception as e:
      #    print(f"I'm a dumb robot and couldn't FUCKING process {note}, because {e}")
  return callback_date_l

In [13]:
def calldate_datetime(df, col):
  calldate_l = []
  for date in df[col]:
    calldatetime = pd.to_datetime(date)
    calldate_l.append(calldatetime)
  return calldate_l

In [ ]:
pd.to_datetime(nullfree_df['Date Called'][2])

Timestamp('2023-09-14 00:00:00')

[Timestamp('2023-09-12 00:00:00'),
 Timestamp('2023-09-13 00:00:00'),
 Timestamp('2023-09-14 00:00:00'),
 Timestamp('2023-09-14 00:00:00'),
 Timestamp('2023-09-15 00:00:00'),
 Timestamp('2023-09-17 00:00:00'),
 Timestamp('2023-09-18 00:00:00'),
 Timestamp('2023-09-18 00:00:00'),
 Timestamp('2023-09-18 00:00:00'),
 Timestamp('2023-09-19 00:00:00'),
 Timestamp('2023-09-19 00:00:00'),
 Timestamp('2023-09-19 00:00:00'),
 Timestamp('2023-09-19 00:00:00'),
 Timestamp('2023-09-19 00:00:00'),
 Timestamp('2023-09-19 00:00:00'),
 Timestamp('2023-09-20 00:00:00'),
 Timestamp('2023-09-20 00:00:00'),
 Timestamp('2023-09-20 00:00:00'),
 Timestamp('2022-09-21 00:00:00'),
 Timestamp('2023-09-25 00:00:00'),
 Timestamp('2023-09-25 00:00:00'),
 Timestamp('2023-09-26 00:00:00'),
 Timestamp('2023-09-27 00:00:00'),
 Timestamp('2023-09-28 00:00:00'),
 Timestamp('2023-09-28 00:00:00'),
 Timestamp('2020-09-29 00:00:00'),
 Timestamp('2023-09-29 00:00:00'),
 Timestamp('2023-09-29 00:00:00'),
 Timestamp('2023-09-

In [ ]:
if date != 'None':
    calldate_l.append(pd.to_datetime(date))
  else:
    calldate_l.append('None')

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 3)

In [ ]:
nullfree_df['callback_date'] = pull_callback_date(nullfree_df, 'Notes', 'Date Called')
nullfree_df['Date Called Formatted'] = calldate_datetime(nullfree_df, 'Date Called')

In [ ]:
nullfree_df

,Name,Date Called,Date Called Back,Call To:,Contact Info,County,Client Name if Different from Caller,Referral From (Agency/Individual),QLSP?,Non-QLSP Type,Message,Notes,Court Type,Time Spent on Meeting,Intake Form Completed,Conflict Check Completed,Resolution,Created By,callback_date
0,Aaron Belcher,9/12/2023,None,Meredith Cell,None,None,None,Fremont private school,No,None,"CPS cops came and grabbed 17 year old and took her phone, at assessment center, probably going to be hosptailized b/c v fragile with anxiety.","9/12 called....9/13 called back, gave info aobut court date, wrote Ginger, passed on his number, wrote DI supervisors with him attached, indicated concerns",None,None,not applicable,not applicable,TA,Anonymous,2024-09-12 00:00:00
1,Melissa Howell,9/13/2023,None,LSPC Main Line,"650-201-6262, Howellfamily015@gmail.com",None,None,Search Engine,No,None,Her husband died in prison. Needs to know what her rights are.,"9/14 Meredith called back after speaking with Kellie, idea is we have Kellie do direct rep for her to get this informatoin and then we make a manual, for how to do it pro se, have to run it by Dorsey. Left her message that said essentially: Hi i'm looking for an attorney, i think i might have an idea, will call you next week. Intake: Husband was transferred to RJ Donovan 12/30/22. He was there 3 months. first was fine. second he got a new celly, but they started bumping heads. A couple other inmates were picking on him. 2018 transferred from Solano to Corcoran. Back in 2019 he got in trouble in Corcoran. Corcoran 2018-2022. Then RJ Donovan. Investigator from Corcoran's nephew was in same housing pod as her husband and approached her husband and said if you give up xyz info, I'll get your visits back. Melissa told him not to give up info. Didn't want him in danger. Said he's entitled to visits and they just needed to wait. Then his celly spoke to CO and CO said sorry we can't move him to new cell. Fast forward Melissa and her daughter saw him March 24 and 25th over spring break 2023. Talked to him when they got home. He contacted her Monday 26th in the morning- said they were being locked down and didn't know how long it would be before they would be able to talk on the phone again.\nThursday that week she got a call he had passed away (4 days after she saw him). She received his stuff. There was a letter he was starting to write saying ""I hope you haven't worried too much. Lock down due to violence of overdoses. COs did cell search, were very rude about it."" She feels like someone put a hot spot on her husband. He had an overdose in Sept of last year- he did have a drug problem. Then in March got weed- said it smelled funny. Said someone did it to him. He had another overdose. Gave him Narcan but was able to walk. Not as bad as in Sept. and he said he was drugged. She has a feeling that they're going to tell her his death was an accidental overdose. But she said he was doing really well since Sept. scare. In visits he was looking really good. She thinks someone intentionally did this.\nWants to know- what's the protocol for cell searches? Do they search both cell and the inmates. Wondering if CO might have passed her husband drugs. Does she need a lawyer to find this out? Feels his celly had a part in this and maybe the other person had a tiff with. Celly was moved to higher level prison in the middle of the investigation. Wants to know if they gave him CPR. Wants camera footage of cell checks that morning. Wants to know what questions she's allowed to ask the prison. Texted on 9/20 to say meeting with Dorsey on Monday",None,None,not applicable,not applicable,information or referral,Arielle Reisman,2024-09-14 00:00:00
2,Brigitte Nicoletti,9/14/2023,None,"Arielle Cell,Meredith Cell",colleague,None,Tahtanerrisha Anecce Sessoms-Howell (Tahti),EBCLC,Yes,None,Brigitte has potential client referral.,Client currently living in hotel on San Pablo- pregnant. SUD in past but very adamant that it's no longer an issue.

In [ ]:
callback_date_l = []
yearpattern = r'\d{4}'
datepattern_old = r'\d{1,2}/\d{1,2}'
datepattern = r'\b(0?[1-9]|1[0-2])/(0?[1-9]|[12][0-9]|3[01])\b'
for note in nullfree_df['Notes']:
  #  try:
    #need to if/then for if it's empty empty
  datefind = re.search(datepattern, note)
  if datefind:
    callbackdate = datefind.group()
    for date in nullfree_df['Date Called']:
      yearfind = re.search(yearpattern, date)
    wholedate = (callbackdate + "/" + yearfind.group())
    final_datetime = pd.to_datetime(wholedate)
    callback_date_l.append(final_datetime)
    #except Exception as e:
    #    print(f"I'm a dumb robot and couldn't FUCKING process {note}, because {e}")
callback_date_l
#print(datefind)

[Timestamp('2024-09-12 00:00:00'),
 Timestamp('2024-09-14 00:00:00'),
 Timestamp('2024-09-18 00:00:00'),
 Timestamp('2024-09-19 00:00:00'),
 Timestamp('2024-09-19 00:00:00'),
 Timestamp('2024-09-19 00:00:00'),
 Timestamp('2024-05-02 00:00:00'),
 Timestamp('2024-09-20 00:00:00'),
 Timestamp('2024-09-20 00:00:00'),
 Timestamp('2024-09-20 00:00:00'),
 Timestamp('2024-09-21 00:00:00'),
 Timestamp('2024-09-21 00:00:00'),
 Timestamp('2024-09-26 00:00:00'),
 Timestamp('2024-09-28 00:00:00'),
 Timestamp('2024-09-28 00:00:00'),
 Timestamp('2024-09-28 00:00:00'),
 Timestamp('2024-09-29 00:00:00'),
 Timestamp('2024-09-29 00:00:00'),
 Timestamp('2024-10-10 00:00:00'),
 Timestamp('2024-09-29 00:00:00'),
 Timestamp('2024-10-03 00:00:00'),
 Timestamp('2024-10-03 00:00:00'),
 Timestamp('2024-10-03 00:00:00'),
 Timestamp('2024-10-05 00:00:00'),
 Timestamp('2024-10-10 00:00:00'),
 Timestamp('2024-10-06 00:00:00'),
 Timestamp('2024-10-08 00:00:00'),
 Timestamp('2024-10-11 00:00:00'),
 Timestamp('2024-10-